In [1]:
import sys

sys.path.append("/Users/rohith_medam/Projects/Portfolio Optimization/Scripts")
import user_def_fncs


In [2]:
import pandas as pd
import numpy as np

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [4]:
import yfinance as yf

In [5]:
import warnings

warnings.filterwarnings("ignore")

In [6]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_rows", 100)

In [7]:
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")

In [8]:
config_params = {
    "Tickers": [
        # Technology
        "AAPL",
        "MSFT",
        "GOOGL",
        "AMZN",
        "TSLA",
        "NVDA",
        "META",
        # Financial
        "BRK-B",
        "JPM",
        "V",
        # Healthcare
        "JNJ",
        "LLY",
        "UNH",
        # Consumer Discretionary
        "PG",
        "WMT",
        "COST",
        # Energy
        "XOM",
        "CVX",
        # Benchmark
        "SPY",
    ],
    "start_date": "2015-01-01",
    "end_date": "2026-01-01",
    "risk_free_rate": 4.18,
    # "rebalance_frequency": "quarterly",
    # "transaction_cost": 0.0015,
    # "train_test_split": 0.8,
    # "validation_split": 0.2,
    # "radom_seed": 42,
}

In [9]:
raw_data = user_def_fncs.download_data(
    tickers=config_params["Tickers"],
    start_date=config_params["start_date"],
    end_date=config_params["end_date"],
)

raw_data

Tickers: AAPL, MSFT, GOOGL, AMZN, TSLA, NVDA, META, BRK-B, JPM, V, JNJ, LLY, UNH, PG, WMT, COST, XOM, CVX, SPY


[*********************100%***********************]  19 of 19 completed


Ticker            META                                                \
Price             Open        High         Low       Close    Volume   
Date                                                                   
2015-01-02   78.034911   78.382481   77.161010   77.905807  18177500   
2015-01-05   77.439069   78.700256   76.326836   76.654549  26452200   
2015-01-06   76.694275   77.051771   74.837244   75.621765  27399300   
2015-01-07   76.227534   76.823371   75.294052   75.621765  22045300   
2015-01-08   76.207667   77.687336   75.552249   77.637680  23961000   
...                ...         ...         ...         ...       ...   
2025-12-24  662.530029  668.179993  662.200012  667.549988   5627500   
2025-12-26  668.059998  668.950012  661.320007  663.289978   7133800   
2025-12-29  658.010010  660.250000  654.390015  658.690002   8506500   
2025-12-30  658.690002  672.219971  657.840027  665.950012   9187500   
2025-12-31  664.750000  665.000000  659.440002  660.090027   7940400   

Ticker           BRK-B                                               \
Price             Open        High         Low       Close   Volume   
Date                                                                  
2015-01-02  151.500000  151.600006  148.500000  149.169998  3436400   
2015-01-05  148.809998  149.000000  146.779999  147.000000  4168800   
2015-01-06  147.639999  148.529999  146.110001  146.839996  4116100   
2015-01-07  147.940002  149.139999  147.649994  148.880005  4159100   
2015-01-08  150.600006  151.369995  150.509995  151.369995  4282100   
...                ...         ...         ...         ...      ...   
2025-12-24  500.290009  501.510010  499.250000  501.339996  2000200   
2025-12-26  500.450012  501.559998  496.829987  498.299988  2247600   
2025-12-29  499.200012  501.500000  497.209991  501.049988  3056200   
2025-12-30  500.980011  505.109985  500.329987  503.709991  2897100   
2025-12-31  503.920013  505.890015  502.480011  502.649994  3105200   

Ticker               V                                                \
Price             Open        High         Low       Close    Volume   
Date                                                                   
2015-01-02   60.961370   61.741382   60.755370   61.340958   8389600   
2015-01-05   60.986847   60.989163   59.917512   59.986954  12751200   
2015-01-06   60.130470   60.359611   58.989381   59.600430  11070000   
2015-01-07   59.817979   60.442919   59.588838   60.398941   9346800   
2015-01-08   60.919712   61.364107   60.787779   61.209034  10443200   
...                ...         ...         ...         ...       ...   
2025-12-24  353.380005  355.989990  353.010010  355.140015   2023600   
2025-12-26  355.100006  356.730011  353.709991  355.000000   2017000   
2025-12-29  355.489990  356.549988  353.799988  354.609985   3989900   
2025-12-30  354.000000  354.869995  352.660004  353.619995   3366500   
2025-12-31  353.649994  355.200012  350.690002  350.709991   3503200   

Ticker            NVDA                                                 \
Price             Open        High         Low       Close     Volume   
Date                                                                    
2015-01-02    0.483011    0.486611    0.475333    0.483011  113680000   
2015-01-05    0.483011    0.484451    0.472694    0.474853  197952000   
2015-01-06    0.475573    0.476053    0.459977    0.460456  197764000   
2015-01-07    0.463816    0.467895    0.457817    0.459257  321808000   
2015-01-08    0.464536    0.479412    0.464296    0.476533  283780000   
...                ...         ...         ...         ...        ...   
2025-12-24  187.940002  188.910004  186.589996  188.610001   65528500   
2025-12-26  189.919998  192.690002  188.000000  190.529999  139740300   
2025-12-29  187.710007  188.759995  185.910004  188.220001  120006100   
2025-12-30  188.240005  188.990005  186.929993  187.539993   97687300   
2025-12-31  189.570007  190.559998  18

In [10]:
processed_data = user_def_fncs.process_raw_data(raw_data, config_params["Tickers"])

Processed data:
Prices shape: (2766, 19)
Returns shape: (2765, 19)
Missing values in prices: 0
Missing values in returns: 0


In [11]:
prices = processed_data["prices"]
volume = processed_data["volume"]
returns = processed_data["returns"]

In [12]:
summary_stats = user_def_fncs.summary_stats(
    returns=returns, prices=prices, risk_free_rate=config_params["risk_free_rate"]
)

In [13]:
%%capture

corr_matrix = user_def_fncs.correlation_analysis(returns)
corr_matrix

In [14]:
fig = px.imshow(
    np.round(corr_matrix, 2),
    labels=dict(color="Correlation Value"),
    title="Portfolio Correlation",
    text_auto=True,
    aspect="auto",
)

fig.show()

In [15]:
is_valid = user_def_fncs.data_validation(returns, prices)

No missing values in returns
No missing values in prices
No extreme returns found


In [16]:
user_def_fncs.data_write(
    prices_data=prices,
    returns_data=returns,
    volume_data=volume,
    summary_stats_data=summary_stats,
    folder_name="Data",
    date_format="%Y-%m-%d",
)

Dataframes saved to /Users/rohith_medam/Projects/Portfolio Optimization/Data as .csv files
Dataframes saved to /Users/rohith_medam/Projects/Portfolio Optimization/Data as an excel file
